# 통계 내기

In [ ]:
import datetime
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyproj import Transformer


In [ ]:
acci_df = pd.read_csv("./data/사고지점_통합_2007_2023.csv")

site_df = pd.read_csv("./data/지오코딩_결과.csv")

site_df = site_df[site_df["위도"] > 37.39]

In [ ]:
# 전처리
# 경도위도 => utm-k 좌표로 변환

gcs_to_utmk = Transformer.from_crs("EPSG:4326", "EPSG:5178", always_xy=True)

tmp_x, tmp_y = gcs_to_utmk.transform(site_df["경도"], site_df["위도"])
site_df["utmk_x"] = tmp_x
site_df["utmk_y"] = tmp_y

site_date = site_df["실착공일"].copy()

site_date.loc[site_df["실착공일"].isna()] = site_df.loc[site_df["실착공일"].isna(), "착공예정일"]

site_date = site_date.apply(lambda x: str(int(x)))

site_df["year"], site_df["month"] = site_date.apply(lambda x: int(x[:4]) ), site_date.apply(lambda x: int(x[4:6]))

site_df

In [ ]:
# 전처리
# 경도위도 => utm-k 좌표로 변환

gcs_to_utmk = Transformer.from_crs("EPSG:4326", "EPSG:5178", always_xy=True)

tmp_x, tmp_y = gcs_to_utmk.transform(site_df["경도"], site_df["위도"])
site_df["utmk_x"] = tmp_x
site_df["utmk_y"] = tmp_y

site_date = site_df["실착공일"].copy()

site_date.loc[site_df["실착공일"].isna()] = site_df.loc[site_df["실착공일"].isna(), "착공예정일"]

site_date = site_date.apply(lambda x: str(int(x)))

site_df["year"], site_df["month"] = site_date.apply(lambda x: int(x[:4]) ), site_date.apply(lambda x: int(x[4:6]))

site_df

# (테스트) 모든 공사 지점 시각화

In [ ]:
plt.figure(figsize=[12, 19])

plt.scatter(site_df["utmk_x"], site_df["utmk_y"])
plt.gca().set_aspect("equal")
plt.xlabel("UTM-K X")
plt.ylabel("UTM-K Y")
plt.show()

# (테스트) 기간 설정 후 지도에 뿌리기

In [ ]:
from matplotlib import patches

start_year, start_month = 2021, 1
end_year, end_month = 2021, 12  # inclusive


def mask_date(df, start_year, start_month, end_year, end_month):
    mask = (
        (df["year"] > start_year) | ((df["year"] == start_year) & (df["month"] >= start_month))
    ) & (
        (df["year"] < end_year) | ((df["year"] == end_year) & (df["month"] <= end_month))
    )  # **** inclusive ****
    return mask


selected_acci_df = acci_df[mask_date(acci_df, start_year, start_month, end_year, end_month)]
selected_site_df = site_df[mask_date(site_df, start_year, start_month, end_year, end_month)]


plt.figure(figsize=[12, 19])
ax = plt.gca()

plt.scatter(selected_acci_df["utmk_x"], selected_acci_df["utmk_y"], s=80, c=(0.0, 0, 1.0, 0.6))

plt.scatter(selected_site_df["utmk_x"], selected_site_df["utmk_y"], s=4, c=(1.0, 0, 0, 1.0))

for x, y in selected_site_df[["utmk_x", "utmk_y"]].values:
    circ = patches.Circle((x, y), 100, alpha=0.3, fc="red")
    ax.add_patch(circ)


plt.gca().set_aspect("equal")
plt.xlabel("UTM-K X")
plt.ylabel("UTM-K Y")
plt.show()

# 애니메이션

In [ ]:
from matplotlib.animation import FuncAnimation
from matplotlib import patches


In [ ]:
year_month = [(y, m) for y in range(2007, 2023) for m in range(1, 13)] + [(2023, 1)]

n_frames = len(year_month) - 2
interval = 5000
mp4_name = "광명시_acci_site.mp4"


def mask_date(df, start_year, start_month, end_year, end_month):
    mask = (
        (df["year"] > start_year)
        | ((df["year"] == start_year) & (df["month"] >= start_month))
    ) & (
        (df["year"] < end_year) | ((df["year"] == end_year) & (df["month"] < end_month))
    )  # **** exclusive ****
    return mask


fig = plt.figure(figsize=[24, 15])

xlim = [
    min(acci_df["utmk_x"].min(), site_df["utmk_x"].min()),
    max(acci_df["utmk_x"].max(), site_df["utmk_x"].max()),
]
ylim = [
    min(acci_df["utmk_y"].min(), site_df["utmk_y"].min()),
    max(acci_df["utmk_y"].max(), site_df["utmk_y"].max()),
]


def animate(i):
    start_year, start_month = year_month[i]
    end_year, end_month = year_month[i + 1]  # inclusive

    selected_acci_df = acci_df[
        mask_date(acci_df, start_year, start_month, end_year, end_month)
    ]
    selected_site_df = site_df[
        mask_date(site_df, start_year, start_month, end_year, end_month)
    ]
    plt.subplot(1, 2, 1)
    ax = plt.gca()
    plt.cla()
    plt.scatter(
        selected_acci_df["utmk_x"],
        selected_acci_df["utmk_y"],
        s=60,
        c=[[0.0, 0, 1.0, 0.4]],
    )

    plt.scatter(
        selected_site_df["utmk_x"],
        selected_site_df["utmk_y"],
        s=4,
        c=[[1.0, 0, 0, 1.0]],
    )

    for x, y in selected_site_df[["utmk_x", "utmk_y"]].values:
        circ = patches.Circle((x, y), 200, alpha=0.2, fc="red")
        ax.add_patch(circ)

    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.gca().set_aspect("equal")
    plt.xlabel("UTM-K X")
    plt.ylabel("UTM-K Y")
    plt.title(f"{start_year}-{start_month}", fontsize=48)

    start_year, start_month = year_month[i + 1]
    end_year, end_month = year_month[i + 2]  # inclusive

    selected_acci_df = acci_df[
        mask_date(acci_df, start_year, start_month, end_year, end_month)
    ]
    selected_site_df = site_df[
        mask_date(site_df, start_year, start_month, end_year, end_month)
    ]
    plt.subplot(1, 2, 2)
    ax = plt.gca()
    plt.cla()
    plt.scatter(
        selected_acci_df["utmk_x"],
        selected_acci_df["utmk_y"],
        s=60,
        c=[[0.0, 0, 1.0, 0.4]],
    )

    plt.scatter(
        selected_site_df["utmk_x"],
        selected_site_df["utmk_y"],
        s=4,
        c=[[1.0, 0, 0, 1.0]],
    )

    for x, y in selected_site_df[["utmk_x", "utmk_y"]].values:
        circ = patches.Circle((x, y), 200, alpha=0.2, fc="red")
        ax.add_patch(circ)

    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.gca().set_aspect("equal")
    plt.xlabel("UTM-K X")
    plt.ylabel("UTM-K Y")
    plt.title(f"{start_year}-{start_month}", fontsize=48)


anim = FuncAnimation(fig, animate, frames=n_frames, interval=interval)

anim.save(mp4_name)

# 두개의 달 연달아 그리기

In [ ]:
year_month = [(y, m) for y in range(2007, 2023) for m in range(1, 13)] + [(2023, 1)]

n_frames = len(year_month) - 2
interval = 100
mp4_name = "광명시_acci_site.mp4"
fig_path = "./figs_compare"

os.makedirs(fig_path, exist_ok=True)

def mask_date(df, start_year, start_month, end_year, end_month):
    mask = (
        (df["year"] > start_year)
        | ((df["year"] == start_year) & (df["month"] >= start_month))
    ) & (
        (df["year"] < end_year) | ((df["year"] == end_year) & (df["month"] < end_month))
    )  # **** exclusive ****
    return mask


fig = plt.figure(figsize=[25, 15])

xlim = [
    min(acci_df["utmk_x"].min(), site_df["utmk_x"].min())-200,
    max(acci_df["utmk_x"].max(), site_df["utmk_x"].max())+200,
]
ylim = [
    min(acci_df["utmk_y"].min(), site_df["utmk_y"].min())-200,
    max(acci_df["utmk_y"].max(), site_df["utmk_y"].max())+200,
]


def animate(i):
    start_year, start_month = year_month[i]
    end_year, end_month = year_month[i + 1]  # inclusive

    selected_acci_df = acci_df[
        mask_date(acci_df, start_year, start_month, end_year, end_month)
    ]
    selected_site_df = site_df[
        mask_date(site_df, start_year, start_month, end_year, end_month)
    ]
    plt.subplot(1, 2, 1)
    ax = plt.gca()
    plt.cla()
    plt.scatter(
        selected_acci_df["utmk_x"],
        selected_acci_df["utmk_y"],
        s=60,
        c=[[0.0, 0, 1.0, 0.4]],
    )

    plt.scatter(
        selected_site_df["utmk_x"],
        selected_site_df["utmk_y"],
        s=4,
        c=[[1.0, 0, 0, 1.0]],
    )

    for x, y in selected_site_df[["utmk_x", "utmk_y"]].values:
        circ = patches.Circle((x, y), 500, alpha=0.12, fc="red")
        ax.add_patch(circ)

    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.xlabel("UTM-K X", fontsize=20)
    plt.ylabel("UTM-K Y", fontsize=20)
    plt.tight_layout()
    plt.gca().set_aspect("equal")
    plt.title(f"{start_year}-{start_month}", fontsize=56)


    start_year, start_month = year_month[i + 1]
    end_year, end_month = year_month[i + 2]  # inclusive

    selected_acci_df = acci_df[
        mask_date(acci_df, start_year, start_month, end_year, end_month)
    ]
    selected_site_df = site_df[
        mask_date(site_df, start_year, start_month, end_year, end_month)
    ]
    plt.subplot(1, 2, 2)
    ax = plt.gca()
    plt.cla()
    plt.scatter(
        selected_acci_df["utmk_x"],
        selected_acci_df["utmk_y"],
        s=60,
        c=[[0.0, 0, 1.0, 0.4]],
    )

    plt.scatter(
        selected_site_df["utmk_x"],
        selected_site_df["utmk_y"],
        s=4,
        c=[[1.0, 0, 0, 1.0]],
    )

    for x, y in selected_site_df[["utmk_x", "utmk_y"]].values:
        circ = patches.Circle((x, y), 500, alpha=0.12, fc="red")
        ax.add_patch(circ)

    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.xlabel("UTM-K X", fontsize=20)
    plt.ylabel("UTM-K Y", fontsize=20)
    plt.title(f"{start_year}-{start_month}", fontsize=56)
    plt.tight_layout()
    plt.gca().set_aspect("equal")
    plt.savefig(f"{fig_path}/{start_year}_{start_month}.png")


anim = FuncAnimation(fig, animate, frames=n_frames, interval=interval)

anim.save(mp4_name)

# (테스트용) 공사 하나 잡고 범위 내 6개월동안의 사고 얻어내기.

In [ ]:
tgt_site = site_df.iloc[1000]


def get_acci_range(site_year, site_month, timespan_month=6):
    # return exclusive range
    if timespan_month <= 0:
        end_year = site_year + site_month // 12
        end_month = site_month % 12 + 1
        return site_year, site_month, end_year, end_month
    start_year = site_year + (site_month - timespan_month - 1) // 12
    start_month = (site_month - timespan_month - 1) % 12 + 1
    end_year = site_year + (site_month + timespan_month - 1) // 12
    end_month = (site_month + timespan_month - 1) % 12 + 1
    return start_year, start_month, end_year, end_month


def mask_date(df, start_year, start_month, end_year, end_month, exclusive=False):
    mask = (df["year"] > start_year) | (
        (df["year"] == start_year) & (df["month"] >= start_month)
    )
    if exclusive:
        mask = mask & (
            (df["year"] < end_year)
            | ((df["year"] == end_year) & (df["month"] < end_month))
        )
    else:
        mask = mask & (
            (df["year"] < end_year)
            | ((df["year"] == end_year) & (df["month"] <= end_month))
        )
    return mask


site_year = tgt_site["year"]
site_month = tgt_site["month"]

timespan = 6

start_year, start_month, end_year, end_month = get_acci_range(
    site_year, site_month, timespan_month=timespan
)

former_acci_nearby_df = acci_df[
    mask_date(acci_df, start_year, start_month, site_year, site_month, exclusive=True)
]
latter_acci_nearby_df = acci_df[
    mask_date(acci_df, site_year, site_month, end_year, end_month, exclusive=True)
]

acci_cnt_ts = []

current_y, current_m = start_year, start_month

for i in range(timespan*2):
    current_y, current_m = current_y + (current_m // 12), (current_m%12) + 1
    acci_cnt_ts.append(acci_df[(acci_df["year"]==current_y) & (acci_df["month"]==current_m)].shape[0])

plt.figure(figsize=(12, 6))
plt.grid(True, axis="y", zorder=2)

plt.bar(np.arange(len(acci_cnt_ts))+0.5, acci_cnt_ts, width=0.85, zorder=10)
plt.bar([timespan-3, timespan+3], [sum(acci_cnt_ts[:timespan]), sum(acci_cnt_ts[timespan:])], width= 3, zorder=9, color="skyblue")
plt.axvline(x=timespan, c="red", zorder=3)
plt.axvspan(xmin=timespan, xmax=timespan*2+1, fc=(1.0, 0.0, 0.0, 0.25), zorder=1)
plt.xticks(np.arange(timespan*2+1), np.arange(-timespan,timespan+1))
plt.title("Accident count before/after #n construction")
plt.ylabel("# of Accidents")
plt.xlabel("$\Delta$months")
plt.xlim([-0.6, timespan*2 + 0.6])
plt.show()


# **(메인코드)** 모든 사고마다

In [ ]:
from matplotlib import font_manager
import tqdm
import cv2

In [ ]:
bg_img = plt.imread("./data/bg_img/광명시1_1000m.png")[:, :, :3]

# 색조-채도-명조로 변환
hsvImage = cv2.cvtColor(bg_img , cv2.COLOR_RGB2HSV)
# 채도 낮추기
hsvImage[:, :, 1] *= 0.2
# 역변환
bg_img = cv2.cvtColor(hsvImage, cv2.COLOR_HSV2RGB)
print(bg_img.shape)

offset = [5000, 5000]
scale = 2.0
plt.imshow(bg_img,
           extent=(offset[0], offset[0]+bg_img.shape[1]*scale, offset[1], offset[1]+bg_img.shape[0]*scale,) , )
plt.show()

In [ ]:
fig_path = "./figs_stats"
os.makedirs(fig_path, exist_ok=True)


filtered_site_df = site_df[site_df["year"]>=2008]
radius = 2000 # unit: m
map_pad = radius+750
dot_scale = 15
timespan = 12
font_hangeul = font_manager.FontProperties(fname="./fonts/Pretendard-Regular.ttf", size=20)
font_hangeul_legend = font_manager.FontProperties(fname="./fonts/Pretendard-Regular.ttf", size=12)
font_hangeul_legend2 = font_manager.FontProperties(fname="./fonts/Pretendard-Regular.ttf", size=14)
# xlim = [
#     min(acci_df["utmk_x"].min(), site_df["utmk_x"].min())-200,
#     max(acci_df["utmk_x"].max(), site_df["utmk_x"].max())+200,
# ]
# ylim = [
#     min(acci_df["utmk_y"].min(), site_df["utmk_y"].min())-200,
#     max(acci_df["utmk_y"].max(), site_df["utmk_y"].max())+200,
# ]

bg_img = plt.imread("./data/bg_img/광명시1_1000m.png")[:, :, :3]

# 색조-채도-명조로 변환
hsvImage = cv2.cvtColor(bg_img , cv2.COLOR_RGB2HSV)
# 채도 낮추기
hsvImage[:, :, 1] *= 0.1
hsvImage[:, :, 2] = np.minimum(hsvImage[:, :, 2]*1.02, 255.0)
# 역변환
bg_img = cv2.cvtColor(hsvImage, cv2.COLOR_HSV2RGB)

offset = [935750, 1933100]
scale = 9.2



def get_acci_range(site_year, site_month, timespan_month=6):
    # return start (year, month), end (year, month) (for exclusive range)
    if timespan_month <= 0:
        end_year = site_year + site_month // 12
        end_month = site_month % 12 + 1
        return site_year, site_month, end_year, end_month
    start_year = site_year + (site_month - timespan_month - 1) // 12
    start_month = (site_month - timespan_month - 1) % 12 + 1
    end_year = site_year + (site_month + timespan_month - 1) // 12
    end_month = (site_month + timespan_month - 1) % 12 + 1
    return start_year, start_month, end_year, end_month




def mask_date(df, start_year, start_month, end_year, end_month, exclusive=False):
    mask = (df["year"] > start_year) | (
        (df["year"] == start_year) & (df["month"] >= start_month)
    )
    if exclusive:
        mask = mask & (
            (df["year"] < end_year)
            | ((df["year"] == end_year) & (df["month"] < end_month))
        )
    else:
        mask = mask & (
            (df["year"] < end_year)
            | ((df["year"] == end_year) & (df["month"] <= end_month))
        )
    return mask

def mask_distance(df, x, y, d=1000):
    mask = (df["utmk_x"] - x)**2 + (df["utmk_y"] - y)**2 <= d**2
    return mask

# 공사 총 개월수 계산 (준공일이 없다면 1개월로 취급)
def get_site_duration(site, start_year=None, start_month=None):
    if (start_year is None) or (start_month is None):
        start_year, start_month = site["year"], site["month"]
    site_duration = 1
    if not pd.isna(site["사용승인일"]):
        site_end_year = int(f"{site['사용승인일']:.0f}"[:4])
        site_end_month = int(f"{site['사용승인일']:.0f}"[4:6])
        site_duration = max(1, (site_end_year - start_year)*12 + (site_end_month-start_month) + 1)
    return site_duration


for site_i in tqdm.tqdm(range(filtered_site_df.shape[0])):
    # 그림 그리고자 하는 대상 공사지
    tgt_site = filtered_site_df.iloc[site_i]

    site_year = tgt_site["year"]
    site_month = tgt_site["month"]
    site_x = tgt_site["utmk_x"]
    site_y = tgt_site["utmk_y"]
    
    # 공사 총 개월수 계산 (준공일이 없다면 1개월로 취급)
    site_duration = get_site_duration(tgt_site, site_year, site_month)

    # 공사 시점으로부터 이웃한 n개월 기간을 구한다.
    start_year, start_month, end_year, end_month = get_acci_range(
        site_year, site_month, timespan_month=timespan
    )

    # 이전 n개월에 일어난 모든 사고를 가져온다.
    former_acci_df = acci_df[
        mask_date(acci_df, start_year, start_month, site_year, site_month, exclusive=True)
    ]
    # 그 중 거리상 가까운 모든 사고를 가져온다.  
    former_acci_nearby_df = former_acci_df[mask_distance(former_acci_df, site_x, site_y, d=radius)]

    # (공사년월을 포함하여) 이후 n개월이 일어난 모든 사고를 가져온다.
    latter_acci_df = acci_df[
        mask_date(acci_df, site_year, site_month, end_year, end_month, exclusive=True)
    ]
    # 그 중 거리상 가까운 모든 사고를 가져온다.
    latter_acci_nearby_df = latter_acci_df[mask_distance(latter_acci_df, site_x, site_y, d=radius)]

    # 시간에 따른 범위 내 사고 카운트 (accident count timeseries)
    acci_cnt_ts = []
    current_y, current_m = start_year, start_month
    tgt_df = former_acci_nearby_df
    for i in range(timespan*2):
        # former 범위 벗어나면 latter로 바꾸어주기.
        # if i == timespan:
        if (current_y, current_m) == (site_year, site_month):
            tgt_df = latter_acci_nearby_df
        acci_cnt_ts.append(tgt_df[(tgt_df["year"]==current_y) & (tgt_df["month"]==current_m)].shape[0])
        current_y, current_m = current_y + (current_m // 12), (current_m%12) + 1


    # 지도 그림의 x, y 범위
    xlim = [site_x-map_pad, site_x+map_pad]
    ylim = [site_y-map_pad, site_y+map_pad]

    fig = plt.figure(num=1, figsize=(13, 17), clear=True)
    gs = fig.add_gridspec(2, 1, height_ratios=(10,48))
    axs = gs.subplots()
    
    # 상단 그림:
    # - 시간에 따른 이웃한 사고 카운트
    # - 시간에 따른 이웃한 공사의 개수
    # - 타겟 공사의 범위
    plt.sca(axs[0])
    plt.grid(True, axis="y", zorder=2)

    ###########################################3
    # 시간에 따른 이웃한 사고 카운트 시각화
    plt.bar(np.arange(-timespan, timespan)+0.5, acci_cnt_ts, width=0.85, zorder=20, color="C0")
    # 정확한 수 어노테이션
    for h_i in range(len(acci_cnt_ts)):
        plt.annotate(f"{acci_cnt_ts[h_i]}", [-timespan+h_i+0.5, acci_cnt_ts[h_i]], ha="center", va="bottom", zorder=100)

    # 이전,이후 각각의 범위 내 사고의 합
    before_after = [sum(acci_cnt_ts[:timespan]), sum(acci_cnt_ts[timespan:])]
    # 위의 것 시각화
    plt.bar([-timespan/2, timespan/2], before_after, width=timespan/2, zorder=10, color="skyblue")
    # 정확한 수 어노테이션
    plt.annotate(f"{before_after[0]}", [-timespan/2, before_after[0]], ha="center", va="top", zorder=100)
    plt.annotate(f"{before_after[1]}", [timespan/2, before_after[1]], ha="center", va="top", zorder=100)
    
    # x=0 붉은 선 = 목표 공사의 시간 기준점
    plt.axvline(x=0, c="red", zorder=3)
    # 목표 공사의 범위 표시
    #plt.axvspan(xmin=0, xmax=timespan*2+1, fc=(1.0, 0.0, 0.0, 0.25), zorder=1)
    plt.axvspan(xmin=0, xmax=site_duration, fc=(1.0, 0.3, 0.3), hatch="////", edgecolor=(0.8, 0.2, 0.2), alpha=0.2, zorder=1)

    #####################################
    # 이웃한 공사 정보 시각화
    # 이웃한 공사 필터링
    other_site_df = filtered_site_df[mask_distance(filtered_site_df, site_x, site_y, d=radius)]
    # 그 중 공사 시작 시점이 그리는 범위의 끝보다 이전 (즉, site_date + n개월 시점보다 이전) 공사들만 필터링
    other_site_df = other_site_df[mask_date(other_site_df, 2008, 1, end_year, end_month, exclusive=True)]
    
    # 시간에 따른 (진행중인) 공사 개수 (site count timeseries)
    site_cnt_ts = [0]*(timespan*2)
    
    other_vspans = []
    for other_i in range(other_site_df.shape[0]):
        # 다른 공사지
        other_site = other_site_df.iloc[other_i]
        other_year, other_month = other_site["year"], other_site["month"]
        # 다른 공사지의 공사 기간
        other_duration = get_site_duration(other_site, other_year, other_month)

        # 다른 공사와 타겟 공사의 시간 차이. (ex. -6 = 6개월 이전)
        other_x = (other_year - site_year)*12 + (other_month-site_month)
        
        # 다른 공사의 끝이(x+duration)이 시각화 범위의 시작 (x=-timespan)을 넘으면
        # 시각화 대상이므로 한다. (공사 시작은 이미 이전으로 보장되기 때문에 반드시 시각화 대상에 포함됨.)
        if other_x + other_duration >= -timespan:
            other_vspans.append((other_x, other_x+other_duration))
            for cnter_idx in range(max(-timespan, other_x), min(timespan, other_x+other_duration)):
                site_cnt_ts[timespan+cnter_idx] += 1
    #for x in other_vspans:
    #    plt.axvspan(xmin=x[0], xmax=x[1], fc=(0.35, 1.0, 0.35, 0.15/max(1, *site_cnt_ts)), zorder=0)


    
    ###########################################
    # 공사 개수 plotting
    ax2 = plt.gca().twinx()
    ax2.plot(np.arange(-timespan, timespan)+0.5, site_cnt_ts, color=(0.17, 0.62, 0.17), marker="^",zorder=30)
    #ax2.set_ylabel("# of Const. sites nearby")
    ax2.set_ylabel("범위 내 공사장의 수", fontproperties=font_hangeul, fontsize=11)
    
    _, ymax = ax2.get_ylim()
    ax2.set_ylim(bottom=0.0, top=ymax*1.15)
    # 시간에 따른 이웃한 공사 정확한 숫자 어노테이션
    for cnter_idx in range(timespan*2):
        plt.text(-timespan+cnter_idx+0.5, ymax*1.03, f"{site_cnt_ts[cnter_idx]}",
             ha="center", va="center", c=(0.17, 0.62, 0.17), zorder=100)
    # 이웃한 공사 숫자 label 붙이기
    #plt.text(-timespan, max([sum(acci_cnt_ts[:timespan]), sum(acci_cnt_ts[timespan:])])*1.02, f"# of Const. sites",
    #         ha="right", va="center", c=(0.17, 0.62, 0.17), zorder=1000, bbox={"facecolor":(1.0, 0.0, 0.0, 1.0), "boxstyle":"round", })
    
    #plt.annotate(f"# of Const. sites", [-timespan, ymax*1.03],
    #       ha="right", va="center", c=(0.17, 0.62, 0.17), zorder=100, bbox={"facecolor":(1.0, 1.0, 1.0, 0.75), "boxstyle":"round", })
    plt.annotate(f"공사의 수", [-timespan, ymax*1.03],
            ha="right", va="center", c=(0.17, 0.62, 0.17), zorder=100, bbox={"facecolor":(1.0, 1.0, 1.0, 0.75), "boxstyle":"round", },
            fontproperties=font_hangeul, fontsize=11)
    plt.sca(axs[0])


    
    ############################################
    # 타겟 공사 시작 시간 / 끝 시간 표기, 그외 기타 정보.
    # plt.text(0.49, 0.99, f"Const. start: {tgt_site['year']}{tgt_site['month']:02d}",
    #          ha="right", va="top", transform =plt.gca().transAxes, zorder=11)
    # plt.text(0.99, 0.99, f"Const. End: {tgt_site['사용승인일']:.0f}",
    #          ha="right", va="top", transform =plt.gca().transAxes, zorder=11)
    # plt.xticks(np.arange(-timespan, timespan+1), np.arange(-timespan,timespan+1))
    # plt.title(f"Accident count before/after construction #{site_i}\nradius: {radius}m", fontsize=20)
    # plt.ylabel("# of Accidents nearby")
    # plt.xlabel("$\Delta$months")
    plt.text(0.49, 0.99, f"공사시작: {tgt_site['year']}{tgt_site['month']:02d}",
             ha="right", va="top", transform =plt.gca().transAxes, zorder=11, fontproperties=font_hangeul, fontsize=11)
    plt.text(0.99, 0.99, f"공사종료:"+f"{tgt_site['사용승인일']:.0f}"[:6],
             ha="right", va="top", transform =plt.gca().transAxes, zorder=11, fontproperties=font_hangeul, fontsize=11)
    plt.xticks(np.arange(-timespan, timespan+1), np.arange(-timespan,timespan+1))
    plt.title(f"공사 전/후 사고빈도 비교 - 샘플 #{site_i}\n이웃반경: {radius}m", fontsize=20, fontproperties=font_hangeul)
    plt.ylabel("사고빈도", fontproperties=font_hangeul, fontsize=11)
    plt.xlabel("$\Delta$months")

    
    # x축 범위, y축 범위 여유좀 주기
    plt.xlim([-0.6-timespan, timespan + 0.6])
    _, ymax = plt.gca().get_ylim()
    plt.ylim(top = ymax*1.1)

    # legend 처리. (보이지 않는 vspan 활용)
    #other_vspan = plt.axvspan(xmin=0, xmax=0, fc=(0.35, 0.35, 1.0),label="Other Const. period")
    # tmp_bar = plt.bar([0], [0], width=0.0, color="C0", label="Accidents nearby")
    # tmp_sum_bar = plt.bar([0], [0], width=0.0, color="skyblue", label="Total accident count (Before/After)")
    # other_plot = plt.plot([0], [0], color=(0.17, 0.62, 0.17), marker="^", label="Const. sites nearby")[0]
    # target_vspan = plt.axvspan(xmin=0, xmax=0, fc=(1.0, 0.2, 0.2), hatch="////", edgecolor=(0.8, 0.2, 0.2), alpha=0.4, label="Target Const. period")
    # plt.legend(handles=(tmp_bar, tmp_sum_bar, other_plot, target_vspan),
    #            loc="upper right", bbox_to_anchor=(1.0, -0.1)).set_zorder(100)
    
    tmp_bar = plt.bar([0], [0], width=0.0, color="C0", label="근처에서 발생한 사고")
    tmp_sum_bar = plt.bar([0], [0], width=0.0, color="skyblue", label="공사 전/후 총 사고빈도")
    other_plot = plt.plot([0], [0], color=(0.17, 0.62, 0.17), marker="^", label="이웃한 공사 개수")[0]
    target_vspan = plt.axvspan(xmin=0, xmax=0, fc=(1.0, 0.2, 0.2), hatch="////", edgecolor=(0.8, 0.2, 0.2), alpha=0.4, label=f"샘플 공사의 기간")
    plt.legend(handles=(tmp_bar, tmp_sum_bar, other_plot, target_vspan),
               loc="upper right", bbox_to_anchor=(1.0, -0.1), prop=font_hangeul_legend).set_zorder(100)

    ##############################################
    # 하단 그림: 타겟 공사의 주변 사고/공사
    plt.sca(axs[1])
    plt.imshow(bg_img,
        extent=(offset[0], offset[0]+bg_img.shape[1]*scale, offset[1], offset[1]+bg_img.shape[0]*scale,), alpha=0.5,)
    
    # 이전에 시작한 공사
    former_site_df = site_df[
        mask_date(site_df, start_year, start_month, site_year, site_month, exclusive=True)
    ]
    # 동시간에 시작한 공사
    current_site_df = site_df[
        mask_date(site_df, site_year, site_month, site_year, site_month, exclusive=False)
    ]
    # 이후에 시작한 공사
    latter_site_df = site_df[
        mask_date(site_df, site_year+site_month//12, site_month%12+1, end_year, end_month, exclusive=True)
    ]
    ax = plt.gca()
    plt.scatter(
        former_acci_df["utmk_x"],
        former_acci_df["utmk_y"],
        s=20*dot_scale,
        c=[[0.0, 0, 1.0, 0.3]],
        marker="X",
        zorder=10,
        # label = "Accident before target const.",
        label = "샘플 공사 착공 이전의 사고",
    )
    plt.scatter(
        latter_acci_df["utmk_x"],
        latter_acci_df["utmk_y"],
        s=20*dot_scale,
        c=[[1.0, 0, 0.0, 0.3]],
        marker="X",
        zorder=10,
        # label = "Accident after target const.",
        label = "샘플 공사 착공 이후의 사고",
    )

    plt.scatter(
        former_site_df["utmk_x"],
        former_site_df["utmk_y"],
        s=5*dot_scale,
        c=[[0.0, 0, 1.0, 1.0]],
        edgecolor = [[1.0, 1.0, 1.0, 0.6]],
        linewidths=2,
        zorder=10,
        # label = "Other const. before target const.",
        label = "샘플 공사 착공 이전의 다른 공사",
    )

    plt.scatter(
        current_site_df["utmk_x"],
        current_site_df["utmk_y"],
        s=5*dot_scale,
        c=[[1.0, 0, 0, 1.0]],
        edgecolor = [[1.0, 1.0, 1.0, 0.6]],
        linewidths=2,
        zorder=10,
        # label = "Other const. at the same time",
        label = "샘플 공사와 동시에 착공한 다른 공사",
    )

    plt.scatter(
        [site_x],
        [site_y],
        s=50*dot_scale,
        c=[[1.0, 1.0, 0, 1.0]],
        edgecolor = [[1.0, 0.0, 0.0, 1.0]],
        marker="o",
        zorder=15,
        # label = "Target const.",
        label = "샘플 공사",
    )
    plt.annotate(f"{tgt_site['대지위치']}",
                 xy=[site_x, site_y],
                 xytext=[30, 40],
                 textcoords="offset pixels",
                 fontproperties=font_hangeul,
                 ha="left",
                 arrowprops=dict(
                     arrowstyle="-",
                     connectionstyle="arc,angleA=-90,angleB=0,armA=20,armB=0,rad=0",
                     relpos=(0.25,0.0),),
                 zorder=14,
            )

    plt.scatter(
        latter_site_df["utmk_x"],
        latter_site_df["utmk_y"],
        s=5*dot_scale,
        c=[[0.85, 0.6, 0.0, 1.0]],
        edgecolor = [[1.0, 1.0, 1.0, 0.6]],
        linewidths=2,
        zorder=10,
        # label = "Other const. after target const.",
        label = "샘플 공사 착공 이후 다른 공사",
    )

    for x, y in former_site_df[["utmk_x", "utmk_y"]].values:
        circ = patches.Circle((x, y), radius=radius, fc=(0.0, 0.0, 0.0, 0.0), ec=(0.0, 0.0, 1.0, 0.08))
        ax.add_patch(circ)
    for x, y in current_site_df[["utmk_x", "utmk_y"]].values:
        circ = patches.Circle((x, y), radius=radius, fc=(0.85, 0.0, 0.6, 0.0), ec=(1.0, 0.0, 0.0, 0.12))
        ax.add_patch(circ)
    for x, y in latter_site_df[["utmk_x", "utmk_y"]].values:
        circ = patches.Circle((x, y), radius=radius, fc=(0.0, 0.0, 0.0, 0.0), ec=(0.85, 0.6, 0.0, 0.08))
        ax.add_patch(circ)
    
    circ = patches.Circle((site_x, site_y), radius=radius,
                          fc=(0.0, 0.0, 0.0, 0.00), ec=(1.0, 0.0, 0.4, 0.8), lw=3)
    ax.add_patch(circ)

    plt.legend(loc=4, fontsize=14, prop=font_hangeul_legend2).set_zorder(100)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.xlabel("UTM-K X", fontsize=12)
    plt.ylabel("UTM-K Y", fontsize=12)
    plt.gca().set_aspect("equal")
    plt.title(f"{start_year}-{start_month:02d}~{end_year}-{end_month:02d}\n"
              f"{tgt_site['대지위치']}",
              fontproperties=font_hangeul,
              fontsize=20)

    plt.tight_layout()
    plt.savefig(f"{fig_path}/{site_i}_{site_year}{site_month:02d}.png")

    #plt.show()


# 단순한 빈도 비교

In [ ]:
filtered_site_df = site_df[(site_df["year"]>=2008) & (site_df["year"] <= 2022)]
radius = 2000 # unit: m
map_pad = radius+500
dot_scale = 15
timespan = 12
# xlim = [
#     min(acci_df["utmk_x"].min(), site_df["utmk_x"].min())-200,
#     max(acci_df["utmk_x"].max(), site_df["utmk_x"].max())+200,
# ]
# ylim = [
#     min(acci_df["utmk_y"].min(), site_df["utmk_y"].min())-200,
#     max(acci_df["utmk_y"].max(), site_df["utmk_y"].max())+200,
# ]


def get_acci_range(site_year, site_month, timespan_month=6):
    # return exclusive range
    if timespan_month <= 0:
        end_year = site_year + site_month // 12
        end_month = site_month % 12 + 1
        return site_year, site_month, end_year, end_month
    start_year = site_year + (site_month - timespan_month - 1) // 12
    start_month = (site_month - timespan_month - 1) % 12 + 1
    end_year = site_year + (site_month + timespan_month - 1) // 12
    end_month = (site_month + timespan_month - 1) % 12 + 1
    return start_year, start_month, end_year, end_month


def mask_date(df, start_year, start_month, end_year, end_month, exclusive=False):
    mask = (df["year"] > start_year) | (
        (df["year"] == start_year) & (df["month"] >= start_month)
    )
    if exclusive:
        mask = mask & (
            (df["year"] < end_year)
            | ((df["year"] == end_year) & (df["month"] < end_month))
        )
    else:
        mask = mask & (
            (df["year"] < end_year)
            | ((df["year"] == end_year) & (df["month"] <= end_month))
        )
    return mask

def mask_distance(df, x, y, d=1000):
    mask = (df["utmk_x"]-x)**2+(df["utmk_y"]-y)**2 <= d**2
    return mask



hit = 0
num = 0

for site_i in range(filtered_site_df.shape[0]):
    tgt_site = filtered_site_df.iloc[site_i]

    site_year = tgt_site["year"]
    site_month = tgt_site["month"]
    site_x = tgt_site["utmk_x"]
    site_y = tgt_site["utmk_y"]

    xlim = [site_x-map_pad, site_x+map_pad]
    ylim = [site_y-map_pad, site_y+map_pad]



    start_year, start_month, end_year, end_month = get_acci_range(
        site_year, site_month, timespan_month=timespan
    )

    former_acci_df = acci_df[
        mask_date(acci_df, start_year, start_month, site_year, site_month, exclusive=True)
    ]
    former_acci_nearby_df = former_acci_df[mask_distance(former_acci_df, site_x, site_y, d=radius)]

    latter_acci_df = acci_df[
        mask_date(acci_df, site_year, site_month, end_year, end_month, exclusive=True)
    ]
    latter_acci_nearby_df = latter_acci_df[mask_distance(latter_acci_df, site_x, site_y, d=radius)]


    acci_cnt_ts = []

    current_y, current_m = start_year, start_month
    tgt_df = former_acci_nearby_df
    for i in range(timespan*2):
        if i == timespan:
            tgt_df = latter_acci_nearby_df
        acci_cnt_ts.append(tgt_df[(tgt_df["year"]==current_y) & (tgt_df["month"]==current_m)].shape[0])
        current_y, current_m = current_y + (current_m // 12), (current_m%12) + 1

    hit += sum(acci_cnt_ts[:timespan]) < sum(acci_cnt_ts[timespan:])
    num += 1



print(f"공사가 있을 때 이후에 사고가 더 많은 개수: {hit}")
print(f"공사가 있을 때 이후에 사고가 더 적은 개수: {num-hit}")
